In [171]:
import functools
import operator
import itertools
import math
import sys
import json
import re
from enum import Enum
from collections import deque

data = open("input.txt").read().splitlines()
coords = [tuple([int(x) for x in line.split(',')]) for line in data]
directions = set([
    (1,0,0),
    (0,1,0),
    (0,0,1),
    (-1,0,0),
    (0,-1,0),
    (0,0,-1)
    ]) 
dims = [0,1,2]
def add(a,b):
    return tuple([x + y for x,y in zip(a,b)])
def plane(a,b):
    min_x = min(a[0],b[0])
    max_x = max(a[0],b[0])
    min_y = min(a[1],b[1])
    max_y = max(a[1],b[1])
    return [(x,y) for x in range(min_x, max_x + 1) 
                  for y in range(min_y, max_y + 1)]

class Kind(Enum):
    UNKNOWN = 0
    SOLID = 1
    INTERIOR = 2
    EXTERIOR = 3

class Droplet():
    def __init__(self, coords) -> None:
        self.min = (math.inf,math.inf,math.inf)
        self.max = (0,0,0)
        self.occupied = coords
        self.voxels = {}
        self.surface_area = 0
        for c in coords:
            self.include(c)
        self._fill_external()
        self._compute_surface_area()
        self.debug = False

    def include(self, coord):
        self.min = tuple([min(self.min[x],coord[x]) for x in dims])
        self.max = tuple([max(self.max[x],coord[x]) for x in dims])
        self.voxels[coord] = Kind.SOLID

    def in_box(self, coord):
        return all([coord[x]>=self.min[x] for x in dims]) and \
               all([coord[x]<=self.max[x] for x in dims])

    def _fill_external(self):
        min_x,min_y,min_z = self.min
        max_x,max_y,max_z = self.max

        shell = list(
            [(x,y,max_z) for x,y in plane((min_x,min_y),(max_x,max_y))]+
            [(x,y,min_z) for x,y in plane((min_x,min_y),(max_x,max_y))]+
            [(min_x,y,z) for y,z in plane((min_y,min_z),(max_y,max_z))]+
            [(max_x,y,z) for y,z in plane((min_y,min_z),(max_y,max_z))]+
            [(x,min_y,z) for x,z in plane((min_x,min_z),(max_x,max_z))]+
            [(x,max_y,z) for x,z in plane((min_x,min_z),(max_x,max_z))]
        )
        total_count = (max_x - min_x + 2) * (max_y - min_y + 2) * (max_z - min_z + 2)
        shell_count = len(shell)
        shell_progress = 0

        def print_progress():
            known = len(self.voxels)      
            print(f"total: {known}/{total_count} => {known/total_count * 100:000f}% ; shell: {shell_progress}/{shell_count}")
        
        print_progress()

        for coord in shell:
            shell_progress += 1
            if shell_progress % 100 == 0:
                print_progress()
            if coord in self.voxels:
                continue
            self.voxels[coord] = Kind.EXTERIOR             
            q = deque([coord])
            pending = set()
            while len(q) > 0:
                p = q.popleft()   
                for d in directions:
                    adjacent = add(p, d)
                    if adjacent in self.voxels or not self.in_box(p):
                        continue
                    self.voxels[adjacent] = Kind.EXTERIOR 
                    q.append(adjacent)
        print_progress()

    def _compute_surface_area(self):
        for cell in self.occupied:
            for direction in directions:
                adjacent = add(cell, direction)
                kind = self.get_kind(adjacent)
                if kind == Kind.EXTERIOR:
                    self.surface_area += 1


    def get_kind(self, coord, path = []):
        kind = self.voxels.get(coord, Kind.UNKNOWN) 
        if kind != Kind.UNKNOWN:
            return kind
        if not self.in_box(coord):
            return Kind.EXTERIOR
        return Kind.INTERIOR

box = set(
      [(x,0,0) for x in range(0,5)] + 
      [(x,1,0) for x in range(0,5)] + 
      [(x,2,0) for x in range(0,5)] + 
      [(x,0,1) for x in range(0,5)] + 
      [(x,1,1) for x in range(0,5)] + 
      [(x,2,1) for x in range(0,5)] + 
      [(x,0,2) for x in range(0,5)] + 
      [(x,1,2) for x in range(0,5)] + 
      [(x,2,2) for x in range(0,5)]
   )

box.remove((0,1,1))
box.remove((1,1,1))
box.remove((2,1,1))
box.add((-1,1,0))
box.add((-2,1,0))
box.add((-2,1,1))

droplet = Droplet(coords)

print(droplet.surface_area)

total: 2117/8820 => 24.002268% ; shell: 0/2320
total: 8810/8820 => 99.886621% ; shell: 100/2320
total: 8810/8820 => 99.886621% ; shell: 200/2320
total: 8810/8820 => 99.886621% ; shell: 300/2320
total: 8810/8820 => 99.886621% ; shell: 400/2320
total: 8810/8820 => 99.886621% ; shell: 500/2320
total: 8810/8820 => 99.886621% ; shell: 600/2320
total: 8810/8820 => 99.886621% ; shell: 700/2320
total: 8810/8820 => 99.886621% ; shell: 800/2320
total: 8810/8820 => 99.886621% ; shell: 900/2320
total: 8816/8820 => 99.954649% ; shell: 1000/2320
total: 8816/8820 => 99.954649% ; shell: 1100/2320
total: 8816/8820 => 99.954649% ; shell: 1200/2320
total: 8816/8820 => 99.954649% ; shell: 1300/2320
total: 8816/8820 => 99.954649% ; shell: 1400/2320
total: 8816/8820 => 99.954649% ; shell: 1500/2320
total: 8816/8820 => 99.954649% ; shell: 1600/2320
total: 8816/8820 => 99.954649% ; shell: 1700/2320
total: 8816/8820 => 99.954649% ; shell: 1800/2320
total: 8816/8820 => 99.954649% ; shell: 1900/2320
total: 8816/